In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df_train= pd.read_csv('../input/fashionmnist/fashion-mnist_train.csv')

In [3]:
import matplotlib.pyplot as plt

In [4]:
import torch
df_train.head()

In [5]:
labels = torch.Tensor(df_train['label'].values)

labels

In [6]:
classes = {
    0:'T-shirt/top',
    1:'Trouser',
    2:'Pullover',
    3:'Dress',
    4:'Coat',
    5:'Sandal',
    6:'Shirt',
    7:'Sneaker',
    8:'Bag',
    9:'Ankle boot',
}

In [7]:
features = torch.Tensor(df_train.iloc[:,1:].values)
plt.title(classes[labels[150].item()])
plt.imshow(features[150].reshape(28,28))
features

In [8]:
#normalizing the pixel values to 0-1 range
max_value = torch.max(features)
features = features/max_value

In [9]:
#to solve this task i will use just a simple soft max layer
from torch import nn
net = nn.Sequential(nn.Linear(784,256),nn.ReLU(),
                    nn.Linear(256,256),nn.ReLU(),
                    nn.Linear(256,10))
net

In [10]:
def init_weight(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,std=0.01)
        
net.apply(init_weight)

In [11]:
loss = nn.CrossEntropyLoss()

In [12]:
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

In [13]:
#lets construct a iterator with the mini batch
def load_array(data_arrays, batch_size):
    dataset = torch.utils.data.TensorDataset(*data_arrays)
    return torch.utils.data.DataLoader(dataset,batch_size)

batch_size = 256
data_iter = load_array((features,labels),batch_size)
next(iter(data_iter))

In [14]:
num_epochs = 1000

for i in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X) ,y.long())
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels.long())
    print(f'epoch {i + 1}, loss {l:f}')

In [ ]:
torch.argmax(net(features)[0]).item()

In [ ]:
plt.figure(figsize=(10,8))
for i in range(10):
    index = np.random.randint(60000)
    plt.subplot(2,5,i+1)
    plt.title(classes[labels[index].long().item()]+'\n'+classes[torch.argmax(net(features)[index]).item()])
    plt.imshow(features[index].reshape(28,28))

In [ ]:
num_hits = 0
predicts = net(features)
for i in range(60000):
    if(torch.argmax(predicts[i]).item() == labels[i].long().item()):
        num_hits+=1
print(f'train accuracy:{num_hits*100/60000:f}%')

In [ ]:
df_test = pd.read_csv('../input/fashionmnist/fashion-mnist_test.csv')

In [ ]:
labels = torch.Tensor(df_test['label'].values)

In [ ]:
features = torch.Tensor(df_test.iloc[:,1:].values)

In [ ]:
type(df_test.iloc[:,1:].values)

In [ ]:
#normalizing the pixel values to 0-1 range
max_value = torch.max(features)
features = features/max_value

In [ ]:
batch_size = 256
data_iter = load_array((features,labels),batch_size)
next(iter(data_iter))

In [ ]:
num_hits = 0
predicts = net(features)
for i in range(10000):
    if(torch.argmax(predicts[i]).item() == labels[i].long().item()):
        num_hits+=1
print(f'test accuracy:{num_hits*100/10000:f}%')

In [ ]:
plt.figure(figsize=(10,8))
for i in range(10):
    index = np.random.randint(10000)
    plt.subplot(2,5,i+1)
    plt.title(classes[labels[index].long().item()]+'\n'+classes[torch.argmax(net(features)[index]).item()])
    plt.imshow(features[index].reshape(28,28))

In [ ]:
import imageio as img

In [ ]:
im = img.imread('../input/real-image/image(1).jpg')

In [ ]:
real_im = 255- torch.Tensor(im[:,:,0])
real_im = real_im/255
print(classes[torch.argmax(net(real_im.reshape(784))).item()])
plt.imshow(real_im)
#real_im

In [ ]:
im = img.imread('../input/alooooo/image(2).jpg')
real_im = 255- torch.Tensor(im[:,:,0])
real_im = real_im/255
print(classes[torch.argmax(net(real_im.reshape(784))).item()])
plt.imshow(real_im)
#real_im